In [1]:
# tutorial.ipynb 
#   based off not_paying_attention_at_exssv.ipynb
#   shows how to calculate orbit + planet precession frequencies and other relevant quantities 
#   runs using python 3.10.6

# Natalia Guerrero 
# updated 17 December 2025

import numpy as np
import numpy.ma as ma
import math as math
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from astropy import units as u
import astropy.constants as const
from astropy.io import fits, ascii
from astropy.time import Time
import matplotlib.colors as colors
from pylaplace import LaplaceCoefficient
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.table import Table, hstack, vstack, Column

# Workbook for individual systems of planets

For a planet-planet pair in the same system, tinker around with: 
  * $g$, $\alpha$
  * obliquity
  * $\omega_{eq}/n$
  * $\delta \tau$, $\sigma \tau$, $\tau$ 
  * and more!  
  

In [3]:
def get_laplace(x):
    """Calculate Laplace coefficient given a semimajor axis ratio a1/a2"""
    # Laplace coefficient
    laplace = LaplaceCoefficient()
    # result = laplace(a, s, m, p, q)
    b = laplace(x, 1.5, 1, 1, 1)
    return b

## import data

In [4]:
# import the info on these planetes
def unpack_fits(filename):
    """get astropy table of a FITS data file"""
    hdul = fits.open(filename)
    tbl = Table(hdul[1].data)
    hdul.close()
    
    return tbl

# Stars T<4700K, <1 transiting planet; most recent publication for each planet
# some are missing Teq, a - calculated separately using Mstar, P
other_filename = "other_struct_planets_extra_teq.fits" #"other_mdwarf_multis_planets_edits_new.fits"  # from Sarah
kepler_filename = "koi_mdwarf_multis_planets.fits"  # also from Sarah

# corrections
#data['MP'][95] = 309.6   # TOI-1130 c? Big boi.

kepdata = unpack_fits(kepler_filename)
kepdata.sort('PERIOD')   # order each system by period
kepdg = kepdata.group_by(['STAR_NUMBER'])

otherdata = unpack_fits(other_filename)
otherdata.sort('PERIOD') # order each system by period
otherdg = otherdata.group_by(['STAR_NUMBER'])

# combine kepler and non-kepler planet groups
both = vstack([otherdg, kepdg])

dg = both
collection = "Kepler + non-Kepler"

In [5]:
# read in Gaia mags for non-Kepler systems
# Gaia mag (Gmag) is close to the Kepler mag peak and the 666nm filter for KOI-255
# we pulled these magnitudes from Exoplanet Archive on 22 jun 2024 (Katie's wedding day)
magdf = pd.read_csv("non_kepler_candidates_gaia_mag.txt", delimiter=" ")

# 1(a) Start here

  * set `pl_name`
  * set `friend_name` (typically the next most outer planet; if it's the outermost planet, the nearest planet interior to it)
  * set `is_kepler`to `True` if it's a KOI and `False` otherwise
  
<!-- We spend a lot of time thinking about Kepler-138. 
  * Kepler-138 b/KOI 314.03
  * Kepler-138 c/KOI 314.01
  * Kepler-138 d/KOI 314.02
  * (Kepler-138 e/no KOI) --> 

In [6]:
# get info for first planet in system
# pl_name = "Kepler-186 d" #"TOI-836 b" #"LHS 3154 b" 
pl_name = "TRAPPIST-1d" # 314.01 #"TOI-270c" #    #"TOI-270c" # 314.03 # "GJ9827c"
friend_name =  "TRAPPIST-1e" # 314.02 #"TOI-270d"  # # "TOI-270d" # 314.02 #"GJ9827d" #"TOI-836 c" #"LHS 3154 b" 

is_kepler = False

if is_kepler:
    st_id_keyword = 'KEPID'
    pl_id_keyword = 'KOI'
    
    pl_tag = f"{pl_id_keyword}-{str(pl_name)}"

else:
    pl_name = f"{pl_name:<14}"  # add spaces at the end
    st_id_keyword = 'HOSTNAME'
    pl_id_keyword = 'NAME'
    hostmag = magdf[magdf['PLANET'] == pl_name.strip(' ')]['GAIAMAG'].to_numpy()[0]
    pl_tag = f"{pl_name.strip(' ')}"

pl_info = dg[dg[pl_id_keyword] == pl_name]

if is_kepler:
    hostmag = pl_info['KEPMAG']


In [7]:
pl_info

NAME,HOSTNAME,STAR_NUMBER,ROWID,KEPID,KOI,DISPOSITION,PERIOD,PERIOD_ERR,T0,T0_ERR,DEPTH,DEPTH_ERR,DUR,DUR_ERR,B,B_ERR,INC,INC_ERR,A,A_ERR,E,E_ERR,AONR,AONR_ERR,RONR,RONR_ERR,RP,RP_ERR,TEQ,TEQ_ERR,TEFF,TEFF_ERR,LOGG,LOGG_ERR,RSTAR,RSTAR_ERR,MSTAR,AGE,AGE_ERR,STELLAR_PARAMS,VETTING,RA,DEC,GL,GB,KEPMAG,GMAG,RMAG,IMAG,KMAG,HMAG,JMAG,NKOI,DRESSING_TEFF,DRESSING_TEFF_ERR_PLUS,DRESSING_TEFF_ERR_MINUS,DRESSING_RSTAR,DRESSING_RSTAR_ERR_PLUS,DRESSING_RSTAR_ERR_MINUS,DRESSING_MSTAR,DRESSING_MSTAR_ERR_PLUS,DRESSING_MSTAR_ERR_MINUS,DRESSING_DIST,DRESSING_DIST_ERR_PLUS,DRESSING_DIST_ERR_MINUS,DRESSING_RONR,ROTATION,ROTATION_ERR,AMPLITUDE,GALHEIGHT,GALHEIGHT_ERR_PLUS,GALHEIGHT_ERR_MINUS,PROVENANCE,MP
str14,str14,float64,float64,float64,float64,str9,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str162,str1,str16,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,float64
TRAPPIST-1d,TRAPPIST-1,53.0,0.0,0.0,0.0,,4.049219,2.6e-05,2457257.068,0.00067,0.3676,0.0063,0.8145,0.004,0.063,0.063,89.896,0.077,0.02227,0.00019,0.07,-100.0,40.216,0.182,0.06063,0.00052,0.788,0.0,288.0,6.0,2566.0,26.0,0.0,0.0,0.12,0.12,0.08900000154972076,-100.0,-100.0,<a refstr=AGOL_ET_AL__2021 href=https://ui.adsabs.harvard.edu/abs/2021PSJ.....2....1A/abstract target=ref>Agol et al. 2021</a>,,346.62639,-5.0434618,0.0,0.0,12.917,19.6235,17.9963,15.0923,10.296,10.718,11.354,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nexsci,0.37407648899548046


In [8]:
def bjd_to_normal(tt):
    """convert barycentric Julian day (bjd) to something I can read
    
    Input:
        tt (bjd), e.g. 2455010.852785 
    Output:
        string, e.g. '2009-06-28T08:28:00.624'
    
    """

    return Time(tt, format='jd').fits

In [9]:
def get_t0_year(tt):
    """get the actual year that a target was first observed
    
        input:
            tt (bjd), e.g. 2455010.852785
        output:
            y1 (int), e.g. 2009
    """

    bjd_offset = 2454833. 
    
    # Kepler targets don't have the offset, so you have to add it on
    if tt < bjd_offset: 
        tt += bjd_offset  
    
    # pick off year from string, ex. '2018-09-28T04:22:04.800'
    return int(bjd_to_normal(tt).split("-")[0]) 

In [10]:
## all planets ins system 

dg[dg[st_id_keyword] == pl_info[st_id_keyword]]

NAME,HOSTNAME,STAR_NUMBER,ROWID,KEPID,KOI,DISPOSITION,PERIOD,PERIOD_ERR,T0,T0_ERR,DEPTH,DEPTH_ERR,DUR,DUR_ERR,B,B_ERR,INC,INC_ERR,A,A_ERR,E,E_ERR,AONR,AONR_ERR,RONR,RONR_ERR,RP,RP_ERR,TEQ,TEQ_ERR,TEFF,TEFF_ERR,LOGG,LOGG_ERR,RSTAR,RSTAR_ERR,MSTAR,AGE,AGE_ERR,STELLAR_PARAMS,VETTING,RA,DEC,GL,GB,KEPMAG,GMAG,RMAG,IMAG,KMAG,HMAG,JMAG,NKOI,DRESSING_TEFF,DRESSING_TEFF_ERR_PLUS,DRESSING_TEFF_ERR_MINUS,DRESSING_RSTAR,DRESSING_RSTAR_ERR_PLUS,DRESSING_RSTAR_ERR_MINUS,DRESSING_MSTAR,DRESSING_MSTAR_ERR_PLUS,DRESSING_MSTAR_ERR_MINUS,DRESSING_DIST,DRESSING_DIST_ERR_PLUS,DRESSING_DIST_ERR_MINUS,DRESSING_RONR,ROTATION,ROTATION_ERR,AMPLITUDE,GALHEIGHT,GALHEIGHT_ERR_PLUS,GALHEIGHT_ERR_MINUS,PROVENANCE,MP
str14,str14,float64,float64,float64,float64,str9,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str162,str1,str16,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,float64
TRAPPIST-1b,TRAPPIST-1,53.0,0.0,0.0,0.0,,1.510826,6e-06,2457257.55,0.00015,0.7378,0.0064,0.601,0.0018,0.095,0.065,89.728,0.165,0.01154,0.0001,0.081,-100.0,20.843,0.094,0.0859,0.00037,1.116,0.0,400.0,8.0,2566.0,26.0,0.0,0.0,0.12,0.12,0.08900000154972076,0.5,-100.0,<a refstr=AGOL_ET_AL__2021 href=https://ui.adsabs.harvard.edu/abs/2021PSJ.....2....1A/abstract target=ref>Agol et al. 2021</a>,,346.62639,-5.0434618,0.0,0.0,12.917,19.6235,17.9963,15.0923,10.296,10.718,11.354,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nexsci,1.2819652607775829
TRAPPIST-1c,TRAPPIST-1,53.0,0.0,0.0,0.0,,2.421937,1.8e-05,2457258.587,0.00027,0.7123,0.0064,0.7005,0.0022,0.109,0.059,89.778,0.118,0.0158,0.00013,0.083,-100.0,28.549,0.129,0.0844,0.00038,1.097,0.0,342.0,7.0,2566.0,26.0,0.0,0.0,0.12,0.12,0.08900000154972076,0.5,-100.0,<a refstr=AGOL_ET_AL__2021 href=https://ui.adsabs.harvard.edu/abs/2021PSJ.....2....1A/abstract target=ref>Agol et al. 2021</a>,,346.62639,-5.0434618,0.0,0.0,12.917,19.6235,17.9963,15.0923,10.296,10.718,11.354,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nexsci,1.2063741570199653
TRAPPIST-1d,TRAPPIST-1,53.0,0.0,0.0,0.0,,4.049219,2.6e-05,2457257.068,0.00067,0.3676,0.0063,0.8145,0.004,0.063,0.063,89.896,0.077,0.02227,0.00019,0.07,-100.0,40.216,0.182,0.06063,0.00052,0.788,0.0,288.0,6.0,2566.0,26.0,0.0,0.0,0.12,0.12,0.08900000154972076,-100.0,-100.0,<a refstr=AGOL_ET_AL__2021 href=https://ui.adsabs.harvard.edu/abs/2021PSJ.....2....1A/abstract target=ref>Agol et al. 2021</a>,,346.62639,-5.0434618,0.0,0.0,12.917,19.6235,17.9963,15.0923,10.296,10.718,11.354,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nexsci,0.37407648899548046
TRAPPIST-1e,TRAPPIST-1,53.0,0.0,0.0,0.0,,6.101013,3.5e-05,2457257.828,0.00041,0.5012,0.0078,0.9293,0.0043,0.191,0.041,89.793,0.048,0.02925,0.0025,0.085,-100.0,52.855,0.239,0.07079,0.00055,0.92,0.0,251.0,5.0,2566.0,26.0,0.0,0.0,0.12,0.12,0.08900000154972076,-100.0,-100.0,<a refstr=AGOL_ET_AL__2021 href=https://ui.adsabs.harvard.edu/abs/2021PSJ.....2....1A/abstract target=ref>Agol et al. 2021</a>,,346.62639,-5.0434618,0.0,0.0,12.917,19.6235,17.9963,15.0923,10.296,10.718,11.354,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nexsci,0.6471670587192556
TRAPPIST-1f,TRAPPIST-1,53.0,0.0,0.0,0.0,,9.20754,3.2e-05,2457257.074,0.00085,0.6465,0.0076,1.048,0.0042,0.312,0.023,89.74,0.019,0.03849,0.00033,0.063,-100.0,69.543,0.314,0.0804,0.00047,1.045,0.0,219.0,4.0,2566.0,26.0,0.0,0.0,0.12,0.12,0.08900000154972076,-100.0,-100.0,<a refstr=AGOL_ET_AL__2021 href=https://ui.adsabs.harvard.edu/abs/2021PSJ.....2....1A/abstract target=ref>Agol et al. 2021</a>,,3


# 1(b) Next stop here

Make sure that the friend name is right. 

Start pulling parameters for the planet + friend from the table. 

In [11]:
# get info for second planet in system

print(f"friend name: {friend_name}")

if not is_kepler:
    friend_name = f"{friend_name:<14}"  # add spaces at the end


# friend_query = NasaExoplanetArchive.query_criteria(
#     table="PS", where=f"pl_name like '{friend_name}'")

# # choose default info for second planet
# friend_info = friend_query[friend_query['default_flag'] == 1]

friend_info = dg[dg[pl_id_keyword] == friend_name]

friend name: TRAPPIST-1e


In [13]:
# get relevant quantities
a = pl_info['A'][0] * u.AU # pl_info['pl_orbsmax']  # semi-major axis, AU
mp =  pl_info['MP'][0] * u.Mearth # pl_info['pl_masse']  # Mearth
mstar = pl_info['MSTAR'][0] * u.Msun # pl_info['st_mass'] # Mstar
P = pl_info['PERIOD'][0] * u.day # pl_info['pl_orbper']   # days
rp = pl_info['RP'][0] * u.Rearth # pl_info['pl_rade']    # Rearth

In [14]:
# is mass missing?
if np.isnan(mp.value): 
    print("mp = NaN")
    print("WARNING! setting mp = rp")
    mp = pl_info['RP'].value * u.Mearth  # mp = rp is an ok approximation
print(f"mp = {mp}")

mp = 0.37407648899548046 earthMass


In [15]:
# is a missing?
if a < 0: 
    print(f"a = {a}")
    print("WARNING! getting a from P")
    # plut in to ye olde Kepler's equation
    a = (((const.G * mstar * P**2) / (4 * np.pi**2))**(1/3)).to(u.AU)
print(f"a = {a}")

a = 0.02227 AU


In [16]:
# get a, P for friend
friend_a = friend_info['A'][0] * u.AU # friend_info ['pl_orbsmax'][0]
friend_P = friend_info['PERIOD'][0] * u.day # friend_info ['pl_orbper'][0]
friend_mp = friend_info['MP'][0] * u.Mearth # friend_info ['pl_masse'][0]


friend_n = ((2 * np.pi) / friend_P).to(u.s**-1)

In [17]:
# is mass missing for friend?
if np.isnan(friend_mp.value): 
    print("friend_mp = NaN")
    print("WARNING! setting mp = rp")
    friend_mp = friend_info['RP'].value * u.Mearth
elif (friend_mp.value > 100):
    friend_mp = friend_info['RP'][0].value * u.Mearth
print(f"mp = {friend_mp}")

mp = 0.6471670587192556 earthMass


In [18]:
# is a missing for friend? 
if friend_a < 0: 
    print(f"friend_a = {friend_a}")
    print("WARNING! calculating friend_a from friend_P")
    friend_a = (((const.G * mstar * friend_P**2) / (4 * np.pi**2))**(1/3)).to(u.AU)
print(f"friend_a = {friend_a}")

friend_a = 0.02925 AU


In [54]:
# stop, evaluate, and listen
#  just checking that the two ways of calculating mean motion are close-ish
print(f"{pl_name} mean-motion")  # [rad / yr]
print(f"------------------------")
print(f"n = 2pi / P = {((2 * np.pi * u.rad) / P).to(u.rad * u.yr**-1):.6}")  
print(f"sqrt(Gmstar / a^3)  = {np.sqrt(const.G * mstar * a**-3).to(u.yr**-1):.6}")

TRAPPIST-1d    mean-motion
------------------------
n = 2pi / P = 566.76 rad / yr
sqrt(Gmstar / a^3)  = 564.009 1 / yr


In [22]:
# find out what MMR ratio we're close to
print(f"a2 / a1 = {friend_a / a}")
print(f"P2 / P1 = {friend_P / P}")

a2 / a1 = 1.3134261338123034
P2 / P1 = 1.506713516853497


## 2. calculate $\alpha$ and $g$

$|g|$ is the observable nodal precession (the precession of the orbit)

$$ g_{LL} = -\frac{1}{4}b_{3/2}^{(1)}(\alpha_{12})\alpha_{12}\left(n_1 \frac{m_2}{M_{*} + m_1}\alpha_{12} + n_2\frac{m_1}{M_* + m_2} \right)  $$

where $\alpha_{12} = a_1/a_2 $ and $n_i$ is the mean motion of each planet and $b_{3/2}^{(1)}$ is a Laplace coefficient we solve for using `pylaplace`

$ \alpha $ (alpha) is the spin-axis precession constant, aka the precession frequency of the planet spin axis

$$ \alpha = \frac{1}{2} \frac{M_{*}}{m_p} \left( \frac{R_p}{a} \right)^3 \frac{k_2}{C} \omega $$

where $\omega$ (omega) is the planetary spin frequency, or the rate at which it rotates. For a tidally-locked planet $\omega = n$, meaning the planet rotates once for every orbit. (We refer to $\omega_{eq}$ later in the notebook, but for the purposes of this tutorial, they're the same.)

 We choose a representative value for $k_2$, the Love number,and $C/MR^2$, which is a function of $k_2$. These are propreties most closely related to the planet density/composition, so this is, at best, an educated guess.

When $g \sim \alpha$, they are _commensurate_ and can drive the planet spin axis to high obliquity, $\epsilon$ (epsilon). 

For this to happen, namely, for the planet to get into tidal Cassini equilibrium for Cassini state 2, we require

$$ \eta = \frac{g}{\alpha} < 1 $$



In [58]:
# mean motion
n = ((2 * np.pi) / P).to(u.s**-1) # mean motion, [rad /s], but just [1/s] really

# alpha
k2 = 0.4 # k2, Love number, drawn from random
C = 0.4 # C, moment of inertia
alpha = (0.5 * (mstar / mp) * (rp / a)**3 * (k2/C) * n).decompose()

# g_ll
a12 = a / friend_a 
g = (-0.25 * get_laplace(a12.value) 
 * a12
 * (n*(friend_mp/(mstar + mp))*a12 + friend_n*(mp/(mstar + friend_mp)))).decompose()

# correct for P2/P1 ratios close to 2:1 or 3:2
np.random.seed(123)  # for reproducibility
if ((friend_P/P < 1.55) & (friend_P/P > 1.5)) | ((friend_P/P < 2.05) & (friend_P/P > 2.0)):
    print(f"P2/P1 = {friend_P/P:.4}, g = 0.3-1.0 * gll")
    g = g*(np.random.rand()*(1. - 0.3) + 0.3)
    
print(f"{pl_name} g/alpha = {g/alpha:.5}")
print(f"orbital precession period = {(2 * np.pi / g).to(u.yr):.5}")
print(f"planet spin axis precession period = {(2 * np.pi / alpha).to(u.yr):.5}")

P2/P1 = 1.507, g = 0.3-1.0 * gll
TRAPPIST-1d    g/alpha = 0.32822
orbital precession period = 248.38 yr
planet spin axis precession period = 81.525 yr


The orbital precession period is how long it takes for the orbit angular momentum axis to make one precession. 

The planet spin axis precession period is how long it takes the planet spin axis to make one precession. Earth's spin axis precesses about once every 26,000 years (and this is why the North Star wasn't the North Star in dinosaur times).

We can get the obliquity, $\epsilon$, the angle for the tilt of the planet spin axis (ex. $23^\circ$ for Earth)

$$ cos~\epsilon = \left( \frac{1}{2\alpha_{syn}/|g| ~-~1} \right)^{1/2}  $$

This expression is satisfied $\alpha_{syn} = \alpha$, meaning when $n = \omega$, i.e. the rotation period of the planet ($\omega$) is the same as its orbital period ($n$). 


In [56]:
# calculate oblicky frequired for spin-orbit resonance
obl = np.arccos(np.sqrt(1/(((2 * alpha)/np.abs(g)) - 1))).to(u.deg) 
print(f"obl = {obl:.5}")

obl = 63.699 deg


We can now get the planet rotation frequency, $\omega_{eq}$: 

$$ \frac{w_{eq}}{n} = \frac{N(e)}{\Omega(e)}\frac{2cos~\epsilon}{1 + cos^2\epsilon} $$

$ N(e=0) = \Omega(e=0) = 1$

which simplifies the expression to 

$$ \frac{w_{eq}}{n} = \frac{2cos~\epsilon}{1 + cos^2\epsilon} $$

and convert that to the length of a "day" for the planet: $T_{sidereal}$ and $T_{solar}$. 

$T_{sidereal}$ is how long it takes the planet to do one rotation (on Earth, 23.9-ish hours)

$T_{solar}$ is how long it takes a point on the planet to rotate all the way around to face the star again (on Earth, 24 hours)

In [ ]:
# calculate weq/n
weqn = (2 * np.cos(np.deg2rad(obl))) / (1 + np.cos(np.deg2rad(obl))**2) 
# weq, equilibrium rotation rate, n = mean-motion of planet

# calculate sidereal day, time to do one rotation
Tsidereal = P / weqn

# calculate solar day, time to face the star
Tsolar = np.abs(Tsidereal / (1 - (Tsidereal/P)))  # days


In [57]:
# stop, evaluate, and listen, part II 
print(f"{pl_id_keyword} {pl_name}")
print(f"--------------------")

print(f"alpha = {alpha:.5}")  
print(f"g     = {g:.5} (paired with {friend_name})")
print(f"alpha / g = {(alpha / g) : .5}")
print(f"eta = |g| / alpha = {(np.abs(g) / alpha) : .5}")
print(f"weqn = {(weqn):.3}")

print(f"obl = {(obl):.3}")
print(f"Teq = {pl_info['TEQ'][0]} K")

print(f"sidereal day = {(Tsidereal):.5}")
print(f"solar day = {(Tsolar):.5}")
print(f"orbital period = {P}")

NAME TRAPPIST-1d   
--------------------
alpha = 2.4422e-09 1 / s
g     = 8.0159e-10 1 / s (paired with TRAPPIST-1e   )
alpha / g =  3.0467
eta = |g| / alpha =  0.32822
weqn = 0.741
obl = 63.7 deg
Teq = 288.0 K
sidereal day = 5.4663 d
solar day = 15.619 d
orbital period = 4.049219 d
